# Init

In [ ]:
from data.datasource.tv_data_feed_api import get_data
from tvDatafeed import TvDatafeed, Interval
import requests

import os
from datetime import datetime
from openpyxl import Workbook, load_workbook
import time
import pandas as pd
import matplotlib.pyplot as plt

import business.utils.trading_signals as ts
import data.datasource.binance_api as ba
LOG_FILE = os.path.join(os.getcwd() , "log.xlsx")
from dotenv import load_dotenv

load_dotenv()


# historical_data = get_data(symbol="BTCUSDT", exchange="BINANCE", interval=Interval.in_1_hour, n_bars=100000)
# historical_data

In [ ]:
def send_notification(message):
    # Replace these with your actual Pushover API token and user key
    app_token = os.getenv('NOTIFICATION_TOKEN')  # e.g., "a1b2c3d4e5f6g7h8i9j0"
    user_key = os.getenv('NOTIFICATION_USER_KEY')    # e.g., "u1v2w3x4y5z6"

    # Prepare the payload with a loud sound ('siren' is one example)
    payload = {
        'token': app_token,
        'user': user_key,
        'message': message,
        'sound': 'siren',  # Change this to any sound of your choice
        'priority': 1      # priority 1 or 2 can trigger an alert mode on some devices
    }
    
    # Send the POST request to Pushover
    response = requests.post("https://api.pushover.net/1/messages.json", data=payload)
    
    if response.status_code == 200:
        print("Notification sent successfully!")
    else:
        print("Failed to send notification. Response:", response.text)


In [ ]:
def log_to_excel(timestamp, symbol, status, message,notified_signal_value,check_func, log_file=LOG_FILE):
    # If the log file doesn't exist, create it with headers
    if not os.path.exists(log_file):
        wb = Workbook()
        ws = wb.active
        ws.title = "Log"
        # Write header row
        ws.append(["Timestamp", "Symbol", "Status", "Message", "Notified Signal Value", "Check Function"])
        wb.save(log_file)
    
    # Load the existing workbook and append a new row
    wb = load_workbook(log_file)
    ws = wb.active
    # If check_func is a function, log its name
    func_name = check_func.__name__ if callable(check_func) else check_func
    ws.append([timestamp, symbol, status, message, notified_signal_value, func_name])
    wb.save(log_file)


# Live Signal Multiple Stocks

In [ ]:
# List of symbols to monitor
symbols = ["BTCUSDT", 
           #"ETHUSDT"
           ]

check_signals = [ts.very_low_price,
                    ts.very_high_price
                    ]

In [ ]:
def run_checks_and_notify():
    for symbol in symbols:
        # Retrieve data for the symbol (adjust parameters as needed)
        data = ba.get_binance_klines(symbol=symbol, interval='1m', limit=100)
        df = pd.DataFrame(data, columns=[
            "timestamp", "open", "high", "low", "close", "volume",
            "close_time", "quote_asset_volume", "number_of_trades",
            "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
        ])
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')
        """
        get_data(
            symbol=symbol,
            exchange="BINANCE",
            interval=Interval.in_1_hour,
            n_bars=1000
        )
        """
        df['open'] = df['open'].astype(float)
        df['high'] = df['high'].astype(float)
        df['low'] = df['low'].astype(float)
        df['close'] = df['close'].astype(float)
        df['volume'] = df['volume'].astype(float)
        # print(df.info())
        # Run each check function on the data
        for check_func in check_signals:
            if not os.path.exists(LOG_FILE):
                wb = Workbook()
                ws = wb.active
                ws.title = "Log"
                # Write header row
                ws.append(["Timestamp", "Symbol", "Status", "Message", "Notified Signal Value", "Check Function"])
                wb.save(LOG_FILE)

            # Now load the log file into a DataFrame
            log_df = pd.read_excel(LOG_FILE)
            last_valid_row = log_df[
                (log_df['Symbol'] == symbol) &
                (log_df['Check Function'] == check_func.__name__) &
                (log_df['Status'])
            ].tail(1)
            status, message, notified_signal_value = check_func(symbol, df, last_valid_row)
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_to_excel(current_time, symbol, status, message,notified_signal_value,check_func.__name__)
            if status:
                send_notification(message)


In [ ]:
run_checks_and_notify()

In [ ]:
df

In [ ]:
log_df = pd.read_excel(LOG_FILE)
last_valid_row = log_df[
    (log_df['Symbol'] == 'ETHUSDT') &
    # (log_df['Check Function'] == '') &
    (log_df['Status'] == False)
].tail(1)

In [ ]:
last_valid_row['Symbol'].values[0]